In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# Mounting Google Drive and Setting up dataset from Kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import os

# changing the working directory
%cd /content/gdrive/My Drive/Normalized

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1NstkfsDWmtDt6AM0JqALnpB2dcIm3Z1u/Normalized


# Importing required Libraries

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 39kB/s 


In [ ]:
!nvidia-smi

Sun Apr 18 07:43:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    30W /  70W |    222MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# importing required libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2 , preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
train_path = "/content/gdrive/MyDrive/Normalized/train"
valid_path = "/content/gdrive/MyDrive/Normalized/test"

opt = Adam(learning_rate = 0.001, beta_1 = 0.9, epsilon = 1e-07, amsgrad = False, name = "Adam")

# Image Augmentation
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path, target_size = (150, 150), batch_size = 32, class_mode = 'categorical')

Found 38478 images belonging to 2 classes.


In [ ]:
test_set = test_datagen.flow_from_directory(valid_path, target_size = (150, 150), batch_size = 32, class_mode = 'categorical')

Found 9618 images belonging to 2 classes.


# Functions

In [ ]:
def set_untrainable(model):
  for layer in model.layers[:-5]:
    layer.trainable = False

def build_model(model):
  # add average global pooling layer, Dropout layer, 1 fully connected layer, and a softmax classifier
  gap = GlobalAveragePooling2D()(model.layers[-5].output)
  do = Dropout(0.5)(gap) 
  fcl = Dense(2, activation = 'softmax')(do)

  # define new model
  model = Model(inputs = model.inputs, outputs = fcl)

  set_untrainable(model)
  return model


def cm_cr(model):
  global test_set, training_set
  test_set.reset()
  Y_pred = model.predict(test_set, len(test_set) //32 + 1)
  y_pred = np.argmax(Y_pred, axis = 1)
  print("Confusion Matrix")
  print(confusion_matrix(test_set.classes, y_pred))

  print("Classification Report")
  target_names = list(training_set.class_indices.keys())
  print(classification_report(test_set.classes, y_pred, target_names = target_names))

def classify(image, model):
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  pred = model.predict(image)
  if pred[0][0] > pred[0][1]:
    print("0 --> Non IDC")
  else:
    print("1 --> IDC")

In [ ]:
%cd /content/gdrive/My Drive/savedModels

/content/gdrive/My Drive/savedModels


# inceptionresnetv2

In [ ]:
inceptionresnetv2 =  InceptionResNetV2(input_shape = (150, 150, 3), weights = 'imagenet', include_top = False)
inceptionresnetv2 = build_model(inceptionresnetv2)
inceptionresnetv2.summary()

219062272/219055592 [==============================] - 4s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
___________________________

In [ ]:
# Checkpoint and Early Stoppng
mcp_save = ModelCheckpoint('irv2', save_best_only = True, monitor = 'val_accuracy', mode = 'max')
early_stopping_monitor = EarlyStopping(monitor = 'val_accuracy', mode = 'max', restore_best_weights = True, patience =  10 )

In [ ]:
# RUNNING FIRST TIME
inceptionresnetv2.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
h = inceptionresnetv2.fit(training_set, validation_data = test_set, epochs = 100, steps_per_epoch = len(training_set) // 32, 
                  validation_steps = len(test_set) // 32, callbacks=[mcp_save, early_stopping_monitor])

In [ ]:
# RUNNING SECOND TIME ONWARDs
inceptionresnetv2 = load_model('irv2')
set_untrainable(inceptionresnetv2)

h = inceptionresnetv2.fit(training_set, validation_data = test_set, epochs = 100, steps_per_epoch = len(training_set) // 32, 
                  validation_steps = len(test_set) // 32, callbacks=[mcp_save, early_stopping_monitor])

# Confusion Matrix And Classification Report

In [ ]:
cm_cr(inceptionresnetv2)

Confusion Matrix
[[1728 3081]
 [1685 3124]]
Classification Report
              precision    recall  f1-score   support

           0       0.51      0.36      0.42      4809
           1       0.50      0.65      0.57      4809

    accuracy                           0.50      9618
   macro avg       0.50      0.50      0.49      9618
weighted avg       0.50      0.50      0.49      9618

